In [37]:
# Import the required libraries
import helpers.MortgagePaymentCalculator as mpc
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import io
from PIL import Image
import os
from datetime import datetime

In [38]:
# Setup widgets using WidgetHelpers
# Display the custom CSS
display(HTML(mpc.WidgetHelpers.style_css()))

# Create the widgets
calculator_widgets = mpc.WidgetHelpers.create_mortgage_payment_calculator_widgets()
# Create the output widgets
summary_output = widgets.Output()


# Define the function to run when the button is clicked
def calculate_and_plot(b):
    # Calculate the mortgage payments
    results = mpc.Calculator.calculate_mortgage_payments(
        calculator_widgets['home_price']['slider'].value,
        calculator_widgets['down_payment_percent']['slider'].value,
        calculator_widgets['loan_term_years']['slider'].value,
        calculator_widgets['interest_rate_exact']['slider'].value,
        calculator_widgets['interest_rate_start']['slider'].value,
        calculator_widgets['interest_rate_end']['slider'].value,
        calculator_widgets['interest_rate_step']['slider'].value,
        calculator_widgets['cash_savings']['slider'].value,
        calculator_widgets['moving_cost']['slider'].value,
    )
    with summary_output:
        summary_output.clear_output(wait=True)
        fig, ax = mpc.Plotter.construct_summary_graphs(results)
        plt.show()

# Define the function to export the current settings and summary to an image
def export_to_image(b):
    results = mpc.Calculator.calculate_mortgage_payments(
        calculator_widgets['home_price']['slider'].value,
        calculator_widgets['down_payment_percent']['slider'].value,
        calculator_widgets['loan_term_years']['slider'].value,
        calculator_widgets['interest_rate_exact']['slider'].value,
        calculator_widgets['interest_rate_start']['slider'].value,
        calculator_widgets['interest_rate_end']['slider'].value,
        calculator_widgets['interest_rate_step']['slider'].value,
        calculator_widgets['cash_savings']['slider'].value,
        calculator_widgets['moving_cost']['slider'].value,
    )
    fig, ax = mpc.Plotter.construct_summary_graphs(results)
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    img = Image.open(buf)

    # Create the exports directory if it doesn't exist
    export_dir = 'exports'
    if not os.path.exists(export_dir):
        os.makedirs(export_dir)

    # Create a timestamped filename
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = os.path.join(export_dir, f'{timestamp}_mortgage_summary.svg')

    # Save the image with the timestamped filename in the exports directory
    fig.savefig(filename, format='svg')
    print(f"Image saved as {filename}")

# Create the export button
export_button = widgets.Button(description="Export to Image")
export_button.on_click(export_to_image)


# Make the sliders update trigger recalculation with replotting
def update_slider(change):
    if change['type'] == 'change' and change['name'] == 'value':
        calculate_and_plot(None)


for widget in calculator_widgets.values():
    widget['slider'].observe(update_slider)

sliders = []
for key, widget in calculator_widgets.items():
    sliders.append(widgets.HBox([widget['slider'], widget['label']]))

ui = widgets.VBox(sliders)

In [39]:
display(ui)

In [40]:
display(widgets.VBox([summary_output, export_button]))